# Trabajo Práctico 2: Pruebas de validez

Integrantes: 

Luis Paredes 104851

# Enunciado

El objetivo consiste en probar la validez de un programa que calcula $2^{8^{20}}$ módulo
p, utilizando diferentes estrategias y viendo el perfil de costos (tanto en términos de
longitud de la prueba como de tiempo de generación). 

1. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1}=a_n^8$.
2. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1} = a_n^2$
3. Considerar 
    * $a_0 = 2$ 
    * $a_{2n+1}=a_{2n}^2$ 
    * $a_{2n}=a_{2n-1}^4$.
4. Usando 2 columnas.

Tener en cuenta que, dado el grado de las restricciones, se requiere usar un factor de
expansión (blowup) por lo menos tan grande como el grado de las restricciones.

In [1]:
import channel, field, list_utils, merkle, polynomial

import time
from field import FieldElement # Cuerpo Finito
from polynomial import X # Polinomios
from polynomial import interpolate_poly # Interpolacion de Lagrange
from polynomial import prod # Producto de polinomios
from merkle import MerkleTree # Merkle Tree
from channel import Channel # Implements Fiat-Shamir Heuristic

from CP import CP_eval, get_CP # Returns a list of images from the CP
from FRI import FriCommit # Next domain for the FRI operator
from decommitment import decommit_fri # proof algorithm

Para todo el trabajo usaremos
* El primo $p = 3 \cdot 2^{30} + 1 = 3221225473$
* Espacio Finitamente generado $F _{3 \cdot 2^{30} + 1}= F_{3221225473}$

# Afirmacion a probar

Quiero probar que conozco el valor de $2^{8^{20}}$ módulo p

 * $a_0 = 2$ 
 * $a_{n+1}=a_n^8$.

# 1. Genero la traza de ejecucion

Generamos unos cuantos valores de la tabla para tener una idea de como se ve

| a_i         | Valor     | 
|--------------|-----------|
| $a_0$ | 2      | 2        |
| $a_1$ | $2^8$ |
| $a_2$      | $2^{64}$ | 
| $a_3$      | $2^{512}$ |

Observamos que $a_1 = 2^{8^1}$ , $a_2 = 2^{8^2}$, $a_3 = 2^{8^3}$ por ende reescribo la ecuacion como

$$a_{n} = a_o^{8^n} = 2^{8^n}, n>0$$

por ende la tabla se vera asi

| $a_i$         | Valor     | 
|--------------|-----------|
| $a_0$        | $2$         |
| $a_1$        | $2^{8^{1}}$ | 
| $a_2$        | $2^{8^{2}}$ |
| $a_3$        | $2^{8^{3}}$ |
| $...$        | $...$ |
|$a_{21}$      |  $2^{8^{20}}$  |

Genero la tabla de ejecucion para los 22 elementos que necesito

In [2]:
print("Generating the trace")

# Tabla de Ejecucion
a = [FieldElement(2)]

# a_{n} = 2^{8^n}
while len(a) < 22:
    a.append(a[0]**(8**(len(a))))

Generating the trace


# 2. Busco un espacio finitamente generado   

Busco elemento generador $g$ tal que el subgrupo sea de orden n y genere el subgrupo 
$G = {g,g^1, g^2, ... , g^n}$, $G[i] = g^i$

Hint: When 𝑘 divides |𝔽×|, $𝑔^𝑘$ generates a group of size |𝔽×| / 𝑘, 
and the n-th power of some FieldElement 𝑥 can be computed by calling x ** n.

---

Queremos que el orden del subgrupo G sea de 32 porque es la potencia de 2 mas pequeña que excede a la traza.

|𝔽×| / 𝑘 = 32

Sabiendo que |𝔽×| = 3221225473

Tomando $k = 100663296$


Tal que 

|𝔽×| / 𝑘 $\approx 32$.

In [3]:
# Nuestro grupo es de orden 32

# Siendo n el orden del subgrupo
# n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
n = 32
mod_F = FieldElement.k_modulus
k = mod_F // n

g = FieldElement.generator() ** (k)  # Busco g^k
G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32

# 3. Busco un polinomio asociado a la traza y hago interpolacion extendiendo la traza

Ahora con el polinomio asociado, cada elemento de la traza lo veo como una evaluacion hecha del polinomio $f$ sobre
elementos $g$ (del espacio generador). Esto se llama Reed-Solomon error correction code.

Ahora la traza se ve asi

|$f(g^i)$| $a_i$         | Valor     | 
|----    |--------------|-----------|
|$f(g^0)$ | $a_0$        | $2$         |
|$f(g^1)$ | $a_1$        | $2^{8^{1}}$ | 
|$f(g^2)$ | $a_2$        | $2^{8^{2}}$ |
|$f(g^3)$ | $a_3$        | $2^{8^{3}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{21})$ |$a_{21}$      |  $2^{8^{20}}$  |



Evaluamos en un dominio mucho mas grande que G. Tomamos un dominio H que es 8 veces mas grande

 n = 8*32

 |𝔽×| = 3221225473
 
 $k \approx 12582912$

In [4]:
print("We blow up the trace (Extend the trace) 8 times bigger")

n = 8*32
mod_F = FieldElement.k_modulus
k = mod_F // n

w = FieldElement.generator()

h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

eval_domain = [w * x for x in H]

We blow up the trace (Extend the trace) 8 times bigger


# 4. Busco un polinomio asociado y hago interpolacion

In [5]:
f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 5959.83it/s]


# 5. Commitment 

Con la traza extendida, ahora creo las hojas del merkle tree a partir de estos 

In [6]:
f_merkle = MerkleTree(f_eval) 

# 6. Channels

Teoricamente el protocolo STARK requiere que dos personas (prover y verifier) interactuen entre si.

En la practica convertimos este protocolo interactivo a uno no interactivo haciendo uso de la heuristica de Fiat-Shamir.

In [7]:
channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
channel.send(f_merkle.root)

El channel va a actuar como nuestro verificador de ahora en adelante. Nos va a proveer con cualquier dato requerido de parte del mismo.

# 7. Generamos CP y FRI layers

Convierto todas las restricciones originales a rational functions.

* $a_0 = 2$

    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $p_o(x) = \frac{f(x) - 2}{x-g^0}$

In [8]:
## a0 = 2
numer0 = f - 2
denom0 = X - 1
p0 = numer0 / denom0

* $a_{n+1}=a_n^8$

    1) $f(g\cdot x) = f(x)^8$
    
    2) $\{g^i \, , 0 \leq i \leq 20 \}$ es raiz de $f(g\cdot x)- f(x)^8$
    
    3) $p_1(x) = \frac{f(g\cdot x)-f(x)^8}{\prod_{i=0}^{20}(x-g^i)}$

In [10]:
# a[n+1] = (an)^8 

numer1 = f(g* X) - f(X)**8
 

lst = [(X - g**i) for i in range(21)]
denom1 = prod(lst)

p1 = numer1 / denom1

Genero el Composition Polynomial como combinacion lineal de estos polinomios.

Los coeficientes del mismo me los provee el channel ya que toma el roll de verificador

In [11]:
cp = get_CP(channel,[p0,p1])
cp_eval = CP_eval(cp,eval_domain)
cp_merkle = MerkleTree(cp_eval)
channel.send(cp_merkle.root)

FRI Folding Operator

In [12]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

# 8. Genero queries y decommitments

In [13]:
len_query = len(eval_domain) - 1
decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)

# 9. Vemos la prueba

In [14]:
print(channel.proof)
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

['send:8bc5d357865faf1f6cc4f21dcfde6012074ada5b899b0e21893ab4bbbfa3e037', 'receive_random_field_element:2934728597', 'receive_random_field_element:1063968342', 'send:2cc554ed6360bd397754815c199367f538b54dae482c1037d63f355064002f4b', 'receive_random_field_element:939738386', 'send:6e96042c0dab68e08619227a338b3b0965541ac4c2dc9b9a41a29cdb477b215a', 'receive_random_field_element:2516983789', 'send:e47ff6a529fc94d30b16641b1cc54c686e7c4181721d7914cde0e8bc0d615578', 'receive_random_field_element:1591322635', 'send:8a38db01c0fdc00d2fd6c371cf15ac45f2d7484b291192810abed763ff661fa4', 'receive_random_field_element:221227579', 'send:83921e52712932d8a4afb31f5729bdbf2edd99496169008dbc07286b8981002f', 'receive_random_field_element:181381405', 'send:7f2938df004427eaf2307ac625c9886cec0d47276fec5dc1535746f5eec7cc92', 'send:-284756853', 'receive_random_int:130', 'send:-1355327771', "send:['633662b2ff311702bb9263d76ed2373498c06d9b6c69abce3663b7eeba38a356', '41df99704195120edbb05baa1460664d8d7ad0c30fcdd7120

Uncompressed proof length in characters: 19178

# Tiempos

Vuelvo a correr toda la prueba e imprimo en pantalla el tiempo que toma correr cada parte

In [ ]:
def modelo_uno():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

    # Tabla de Ejecucion
    a = [FieldElement(2)]

    # a_{n} = 2^{8^n}
    while len(a) < 22:
        a.append(a[0]**(8**(len(a))))

    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0


    # a[n+1] = (an)^8 
    numer1 = f(g* X) - f(X)**8

    lst = [(X - g**i) for i in range(21)]
    denom1 = prod(lst)
    p1 = numer1 / denom1


    cp = get_CP(channel,[p0,p1])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_uno()

Vemos que tomamos
- 0.01599 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.06235 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.3503 segundos en total

Los tiempos suelen variar ligeramente entre compilaciones

# Conclusion

El protocolo STARK es una protocolo que nos permite demostrar o validar el trabajo hecho sin tener que hacerlo de vuelta o entregar datos personales que demuestren lo que afirmamos.

Con las pruebas de validez podemos probar que un cómputo se ha realizado correctamente en un tiempo bastante menor que la verificación trivial, es decir, que el cómputo sea corrido de manera independiente por el verificador. 

Ignorando un momento toda la matematica de trasfondo, el protocolo actua casi como magia y el hecho de que no requiera de interaccion externa le da una seguridad enorme. 

Sin embargo, con estas soluciones tambien creamos nuevos problemas. Como dependemos fuertemente del channel es critico que su aleatoridad sea absoluta y en el mundo real esto es algo que no podemos afirmar a ciegas. Si algun dia se descubre una manera de predecir los valores que nos da entonces todo el protocolo se romperia. 

Pero como todo en criptografia, el protocolo funciona bien hasta que inevitablemente alguien lo rompa. Por el momento, STARK parece ser muy prometedor para el futuro de la criptografia y ya hay planes para incorporarlo como una mejora de su protocolo hermano SNARK.